<a href="https://colab.research.google.com/github/kush1702/AKATSUKI-ADOBEDEVCRAFT/blob/main/fine%20tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv("dataset101.csv",encoding="cp1252")

In [ ]:
df.head()

,text,label
0,"Life in the Deserts\nIn Chapter 5, you have se...",Education Material
1,A Few \nSelections of\nFavorite\nPoems\n(in En...,Creative Writing
2,TERMS OF USE \n\nPage 1 of 3\n\nAvaTrade is ...,Legal Document
3,To whom it may concern:\nElon Musk’s takeover ...,Business Document
4,Printed in Thailand\n6MB30711-01No reproductio...,Technical Document


In [ ]:
print(df)

                                                text  \
0  Life in the Deserts\nIn Chapter 5, you have se...   
1  A Few \nSelections of\nFavorite\nPoems\n(in En...   
2   TERMS OF USE  \n\nPage 1 of 3\n\nAvaTrade is ...   
3  To whom it may concern:\nElon Musk’s takeover ...   
4  Printed in Thailand\n6MB30711-01No reproductio...   
5  File Number (For Office Use Only)\n\n\n\nGOVER...   
6  IMF Annual  \nReport 2022\nFINANCIAL\nSTATEMEN...   
7  International Journal of Research Publication ...   
8  LPL - LPL-ROHINI (NATIONAL REFERENCE \nLAB)\nS...   
9  stem cell researchHealth Research\nStudying st...   

                        label  
0          Education Material  
1            Creative Writing  
2              Legal Document  
3           Business Document  
4          Technical Document  
5         Government Document  
6          Financial Document  
7               News Document  
8            Medical Document  
9  Scientific Research Papers  


In [ ]:
from transformers import AutoModel, AutoTokenizer,AutoModelForSequenceClassification

model_path = 'knowledgator/comprehend_it-base'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
print(list(df['label']))

['Education Material', 'Creative Writing', 'Legal Document', 'Business Document', 'Technical Document', 'Government Document', 'Financial Document', 'News Document', 'Medical Document', 'Scientific Research Papers']


In [ ]:
pip install liqfit

In [ ]:
pip install datasets

In [ ]:
from liqfit.datasets import NLIDataset

# Assuming df is a pandas DataFrame with 'text' and 'label' columns
texts = list(df['text'])
labels = list(df['label'])
classes = list(set(labels))  # Assuming classes are unique labels in the 'label' column

# N is the number of examples you want to take
N = 10

# Take N examples for training
train_texts = texts[:N]
train_labels = labels[:N]
test_texts = texts[:N]
test_labels = labels[:N]

# Create instances of NLIDataset
nli_train_dataset = NLIDataset(train_texts, train_labels, classes)
nli_test_dataset = NLIDataset(test_texts, test_labels, classes)


In [ ]:
pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.5 MB/s eta 0:00:00


In [ ]:
from liqfit.modeling import LiqFitModel
from liqfit.losses import FocalLoss

backbone_model = AutoModelForSequenceClassification.from_pretrained('microsoft/deberta-v3-xsmall')

loss_func = FocalLoss()

model = LiqFitModel(backbone_model.config, backbone_model, loss_func=loss_func)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-xsmall and are newly initialized: ['pooler.dense.bias', 'classifier.weight', 'pooler.dense.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from liqfit.collators import NLICollator
data_collator = NLICollator(tokenizer, max_length=128, padding=True, truncation=True)

In [ ]:
from transformers import TrainingArguments,Trainer
training_args = TrainingArguments(
    output_dir='comprehendo',
    learning_rate=3e-5,
    per_device_train_batch_size=3,
    per_device_eval_batch_size=3,
    num_train_epochs=9,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_steps = 5000,
    save_total_limit=3,
    remove_unused_columns=False,
)

In [ ]:
pip install torch

In [ ]:
from transformers import TrainingArguments, Trainer
import torch

# Assuming train_labels and test_labels are your label lists
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Verify the number of classes
num_classes = len(label_encoder.classes_)
print("Number of Classes:", num_classes)

# Update nli_train_dataset and nli_test_dataset with encoded labels
nli_train_dataset.labels = torch.tensor(train_labels_encoded)  # Convert to tensor
nli_test_dataset.labels = torch.tensor(test_labels_encoded)    # Convert to tensor

# Update model configuration with the correct number of labels
model.config.num_labels = num_classes

# Rest of your Trainer code
training_args = TrainingArguments(
    output_dir='comprehendo',
    learning_rate=3e-5,
    per_device_train_batch_size=3,
    per_device_eval_batch_size=3,
    num_train_epochs=9,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_steps=5000,
    save_total_limit=3,
    remove_unused_columns=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=nli_train_dataset,
    eval_dataset=nli_test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()


In [ ]:
print("Encoded Values for a few examples:", train_labels_encoded[:10])


Encoded Values for a few examples: [2 1 5 0 9 4 3 7 6 8]
